In [1]:
%matplotlib inline
import numpy as np
import cv2
from incisorseg.dataset import Dataset,LeaveOneOutSplitter,load_image,load_landmark
from incisorseg.utils import *
from asm.shape import Shape, ShapeList
from asm.shapemodel import ShapeModel
from asm.greymodel import GreyModel
from asm.model import ActiveShapeModel

In [2]:
data = Dataset('../data/')

In [4]:
i = 0
for split in LeaveOneOutSplitter(data):
    if i > 0:
        break
    training_images,training_landmarks,training_segmentations = split.get_training_set()
    test_image,test_landmark,test_segmentation = split.get_test_example()
    scaled_training_images = training_images
    scaled_training_landmarks = training_landmarks
    scaled_test_image = test_image
    for j in range(4):
        scaled_test_image = cv2.pyrDown(scaled_test_image)
        scaled_training_images = [cv2.pyrDown(image) for image in scaled_training_images]
        scaled_training_landmarks = ShapeList([shape.pyr_down() for shape in scaled_training_landmarks])
        if j==3:
            #print scaled_training_landmarks.bounding_box()
            #print scaled_training_images[0].shape
            model = ActiveShapeModel(scaled_training_images,scaled_training_landmarks,grey_model_number_of_pixels=10,
                 grey_model_search_number_of_pixels=40)
            new_shape_combined,fit_error,_ = model.fit(scaled_test_image,1e-4,20)
            imshow2(overlay_shapes_on_image(scaled_test_image,[new_shape_combined.round()]))
            plot_shapes([new_shape_combined])

    #print split.get_dice_error_on_test(test_landmark)
    #print split.get_dice_error_on_test(model.initial_shape())
    #print split.get_dice_error_on_test(new_shape_combined)
    i +=1

[ 97.990625  52.259375]
[ 89.89375  31.5    ]
[ 86.125     20.728125]
[ 80.2375    13.096875]
[ 80.275   9.475]
[ 81.15625    9.484375]
[ 81.14375   9.91875]


LinAlgError: SVD did not converge